In [1]:
import pandas as pd
import glob
import os

In [11]:
carpeta = "innovation moderate/produccion-energetica"       # Carpeta donde están tus excels de instalaciones
archivo_hdd = "HDD_por_dia_promedio.xlsx"

In [12]:
# --- 1. Leer archivo HDD ---
hdd = pd.read_excel(archivo_hdd, engine="openpyxl")
hdd["Date"] = hdd["Date"].astype(str).str.zfill(5)

In [ ]:
! conda install -c anaconda openpyxl=3.1.5


In [15]:
resultados = []
df_final = pd.DataFrame()

# --- 3. Recorrer todos los Excel de la carpeta ---
for ruta in glob.glob(os.path.join(carpeta, "*.xlsx")):
    nombre_instalacion = os.path.splitext(os.path.basename(ruta))[0]
    print(f"Procesando instalación: {nombre_instalacion}")

    df = pd.read_excel(ruta)
    df.columns = df.columns.str.strip()  # Limpia espacios en nombres

    # Normalizar columnas
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    df["EnergiaCaldera(MW-hr)"] = df["EnergiaCaldera(MW-hr)"].astype(str).str.replace(",", ".").astype(float)

    # Crear columna Date con formato día-mes
    df["Date"] = df["Timestamp"].dt.strftime("%d-%m")

    # Agrupar por día y sacar media (por si hay varios años)
    df_media = df.groupby("Date", as_index=False)["EnergiaCaldera(MW-hr)"].mean()

    # Añadir nombre de instalación
    df_media["Instalacion"] = nombre_instalacion

    # Unir con HDD
    combinado = pd.merge(df_media, hdd, on="Date", how="left")

    # Reordenar columnas
    combinado = combinado[["Date", "Instalacion", "HDD", "EnergiaCaldera(MW-hr)"]]

    resultados.append(combinado)

    # --- 4. Unir todo ---
    df_final = pd.concat(resultados, ignore_index=True)


Procesando instalación: m218807
Procesando instalación: m218820
Procesando instalación: m220825


C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "1 feb 2024 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "2 feb 2024 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead

Procesando instalación: m220850
Procesando instalación: m220853


C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "6 feb 2023 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "7 feb 2023 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead

Procesando instalación: m220854
Procesando instalación: m220860


...

C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "31 jul 2023 00:00:00 CEST" included an un-recognized timezone "CEST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "1 sep 2023 00:00:00 CEST" included an un-recognized timezone "CEST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "2 sep 2023 00:00:00 CEST" included an un-recognized

Procesando instalación: m222820


C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "1 may 2025 00:00:00 CEST" included an un-recognized timezone "CEST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "2 may 2025 00:00:00 CEST" included an un-recognized timezone "CEST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "3 may 2025 00:00:00 CEST" included an un-recognized 

Procesando instalación: m222821
Procesando instalación: m222824
Procesando instalación: m223296


C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "4 mar 2024 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
C:\Users\aguer\AppData\Local\Temp\ipykernel_18772\2549576052.py:13: FutureWarning: Parsed string "5 mar 2024 00:00:00 CET" included an un-recognized timezone "CET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead

In [16]:
print(df_final.head(10))

    Date Instalacion      HDD  EnergiaCaldera(MW-hr)
0  01-02     m218807   7.2750                 220.30
1  01-03     m218807   4.8875                 238.10
2  01-05     m218807  11.0500                 266.25
3  01-06     m218807  16.2250                 273.05
4  01-07     m218807  18.0000                 212.90
5  01-11     m218807  12.8500                 224.80
6  02-02     m218807   7.1250                 220.95
7  02-03     m218807   5.4750                 238.75
8  02-05     m218807  11.9750                 266.65
9  02-06     m218807  17.0250                 211.20


In [17]:
salida = os.path.join("base_datos_energia_hdd.xlsx")
df_final.to_excel(salida, index=False)